



The issue of class imbalance can result in a serious bias towards the majority class, reducing the classification performance and increasing the number of false nagatives. How can we alleviate the issue? The most commonly used techniques is data resampling either under-sampling the majority of the class, or over-sampling the minority class, or a mix of both. This will result in an improved classification performance. In this article I will explain what is imbalanced data, why ROC fails to measure correctly, and the techniques to attack the issue. For each method, I include the Python code for those who are interested. 
        
### What is imbalanced data?

The definition of imbalanced data is straightforward. A dataset is imbalaned if at least one of the classes constitutes only a very small minority.  Imbalanced data prevail in banking, insurance, engineering and many other fields. It is common in fraud detection that the imbalance is on the order of 100 to 1.

###  Why an ROC curves cannot measure well

The Receiver operating characteristic (ROC) curve is the typical tool for assessing the performance of machine learning algorithms, but it actually does not measure well for imbalanced data. Consider an insurance company who needs to know a claim is fraudulent or not. The company needs to predict claims that may be fraudulent and take steps to prevent. Suppose 2% of 10,000 claims are fraudulent. If the data scientist predicts that ALL claims are not fraudulent, he gets 98% accuracy in the ROC curve. However, the data scientist totally missed the 2% true fraudulent claims.

Let's formulate this decision problem with the labels as either positive or negative. The performance of a model can be represented in a *confusion matrix* with four categories. True positives (TP) are positive examples that are correctly labeled as positives, and False positives (FP) are negative examples that are labeled incorrectly as positive. Likewise, True negatives (TN) are negatives labeled correctly as negative, and false negatives (FN) refer to positive examples labeled incorrectly as negative.

<table style="width:40%">
  <tr>
    <th></th>
    <th>Actual positive</th> 
    <th>Actual negative</th>
  </tr>
  <tr>
    <td>Predicted positive</td>
    <td>TP</td>
    <td>FP</td>
  </tr>
  <tr>
    <td>Predicted negative</td>
    <td>FN</td>
    <td>TN</td>
  </tr>
</table>

Given the confusion matrix, we are able to define the matrics:

<table style="width:40%">
  <tr>
    <th>Name</th>
    <th>Metric</th>
  </tr>
  <tr>
  <tr>
    <td>True Positive Rate (TPR)</td>
    <td>TP / (TP+FN)</td>
  </tr>
  <tr>
    <td>False Positive Rate (FPR)</td>
    <td>FP / (FP+TN)</td>
  </tr>
    <td>Recall</td>
    <td>TP / (TP+FN)</td>
  </tr>
  <tr>
    <td>Precision</td>
    <td>TP / (TP+FP)</td>
  </tr>
</table>

The Receiver Operating Characteristic (ROC) curves plot FPR vs. TPR as shown below. Because TPR only depends on positives, ROC curves do not measure the effects of negatives. The area under the ROC curve (AUC) assesses overall classification performance. AUC does not place more emphasis on one class over the other, so it does not reflect the minority class well.

<img src="images/ROC_PR_for_ROC.png" alt="Drawing" style="width: 400px;"/>

Davis and Goadrish in a well-cited [paper](http://machinelearning.wustl.edu/mlpapers/paper_files/icml2006_DavisG06.pdf) propose that Precision-Recall (PR) curves will be more informative than ROC  when dealing with highly skewed datasets. The PR curves plot precision vs. recall (FPR). Because Precision is directly influenced by class imbalance so the Precision-recall curves are better to highlight differences between models for highly imbalanced data sets. When you compare different models with imbalanced settings, area under the Precision-Recall curve will be more sensitive than the area under the ROC curve.

<img src="images/ROC_PR_for_PR.png" alt="Drawing" style="width: 400px;"/>

### What are the remedies?

Broadly speaking there are three major approaches to handle imbalanced data: data sampling, algorithm modifications and cost-sensitive learning. In this chapter I will focus on the data sampling techniques.

### Using undersampling techniques

###### (1) Random undersampling for the majority class

A simple undersampling technique is to undersample the majority class randomly and uniformly. This can potentially lead to loss of information. But if the examples of the majority class are near to others, this method might yield good results.

###### (2) NearMiss

In order to attack the issue of potential information loss, "near neighbor" method and its variations have been proposed. The basic algorithms of the near neighbor family is this: first the method calculates the distances between all instances of the majority class and the instances of the minority class. Then k instances of the majority class that have the smallest distances to those in the minority class are selected. If there are n instances in the minority class, the "nearest" method will result in k*n instances of the majority class.

"NearMiss-1" selects samples of the majority class that their average distances to three closest instances of the minority class are the smallest. "NearMiss-2" uses three farthest samples of the minority class. "NearMiss-3" selects a given number of the closest samples of the majority class for each sample of the minority class.  

###### (3) Condensed Nearest Neighbor Rule (CNN)

To avoid losing potentially useful data, some heuristic undersampling methods have been proposed to remove redundant instances that should not affect the classification accuracy of the training set. Hart (1968) introduced the Condensed Nearest Neighbor Rule (CNN). Hart starts with two blank datasets A and B. Initially the first sample is placed in dataset A, while the rest samples are placed in dataset B. Then one instance from dataset B is scanned by using dataset A as the training set. If a point in B is misclassified, it is transferred from B to A. This process repeats until no points are transferred from B to A. 


###### (4) TomekLinks

In the same manner, Tomek (1976) proposed an effective method that consider samples hear the borderline. Given two instances a and b belonging to different classes and are separated by a distance d(a,b), the pair (a, b) is called a Tomek link if there is no
instance c such that d(a,c) < d(a,b) or d(b,c) < d(a,b). Instances participating in Tomek links are either borderline or noise so both are removed.

###### (5) Edited Nearest Neighbor Rule (ENN)

Wilson (1972) introduced the Edited Nearest Neighbor Rule (ENN) to remove any instance whose class label is different from the class of at least two of its three nearest neighbors. The idea behind this technique is to remove the instances from the majority class that are near or around the borderline of different classes based on the concept of nearest neighbor (NN) in order to increase classification accuracy of minority instances rather than majority instances.

###### (6) NeighbourhoodCleaningRule

Neighborhood Cleaning Rule (NCL) deals with majority and minority samples separately when sampling the data sets. NCL uses ENN to remove majority examples. for each instance in the training set, it finds three nearest neighbors. If the instance belongs to the majority class and the classification given by its three nearest neighbors is the opposite of the class of the chosen instance, then the chosen instance is removed. If the chosen instance belongs to the minority class and is misclassified by its three nearest neighbors, then the nearest neighbors that belong to the majority class are removed.


###### (7) ClusterCentroids

This method undersamples the majority class by replacing a cluster of majority samples 
This method finds the clusters of majority class with K-mean algorithms. Then it keeps the cluster centroids of the N clusters as the new majority samples.


###### R code

Below I demonstrate the sampling techniques with the Python scikit-learn module [imbalanced-learn](http://contrib.scikit-learn.org/imbalanced-learn/stable/install.html).
Readers need to install the Python package. The following data generation progress (DGP) generates 2,000 samples with 2 classes. The data is extremely umbalanced with the proportion 0.03 and 0.97 assigned to each class. There are 10 features, of which 2 are informative and 2 are redundant and 6 (10-2-2) are repeated features. The make_classification function generates the repeated (useless) features from the informative and the redundant features. The redundant features are simply the linear combinations of the informative features. Each class is consisted of 2 gaussian clusters. For each cluster, informative features are drawn independently from N(0, 1) and then linearly combined together within each cluster. It is important to know if the parameter *weights* is left blank, then classes are balanced. 

